<a href="https://colab.research.google.com/github/Tabrid/iinms/blob/main/water_requirement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import math

T_max=24.6000003814697
T_min=11.5
T_mean=(T_max + T_min) / 2
Humidity=77.6666666666667
SSH=7.80000019073486
WS=1.36694057142857
Julian_Day=228
Latitude=24.4
lat_rad = math.radians(Latitude)

In [26]:
import math

def calculate_Es(T_mean):
    return 0.6108 * math.exp((17.27 * T_mean) / (237.3 + T_mean))

In [27]:
def calculate_delta(T_mean):
    ES_value = calculate_Es(T_mean)
    delta = (4098 * ES_value) / ((237.3 + T_mean) ** 2)
    return delta

In [28]:
delta_value=calculate_delta(T_mean)

In [29]:
import math

def e_tmin(tmin: float) -> float:
    return 0.6108 * math.exp((17.27 * tmin) / (tmin + 237.3))
def e_tmax(tmax: float) -> float:
    return 0.6108 * math.exp((17.27 * tmax) / (tmax + 237.3))
calculate_e_tmin = e_tmin(T_min)
calculate_e_tmax = e_tmin(T_max)
calculate_es = (calculate_e_tmin + calculate_e_tmax) / 2
calculate_ea = calculate_es * (Humidity / 100)


print(calculate_e_tmin)
print(calculate_e_tmax)
print(calculate_es)
print(calculate_ea)

1.3569857803790661
3.093081400019516
2.225033590199291
1.72810942172145


In [30]:
def delta(Julian_Day: int) -> float:
    return 0.409 * math.sin((2 * math.pi * Julian_Day / 365) - 1.39)


def ws_hs() -> float:
  delta_result = delta(Julian_Day)
  return math.acos(-math.tan(lat_rad) * math.tan(delta_result))

calculate_delta = delta(Julian_Day)
calculate_ws_hs = ws_hs()

In [31]:
import math

def calculate_n(calculate_ws_hs):
    return (24 / math.pi) * calculate_ws_hs

calculate_n = calculate_n(calculate_ws_hs)
nM = SSH/calculate_n

def calculate_rb(nM, calculate_ea, T_mean):
    return (0.1 + 0.9 * nM) * (0.34 - 0.14 * math.sqrt(calculate_ea)) * 4.9e-9 * (273 + T_mean) ** 4

calculate_rb = calculate_rb(nM, calculate_ea, T_mean)
print(calculate_rb)

3.5500667475175645


In [32]:
def calculate_dr(Julian_Day):
    return 1 + 0.033 * math.cos(2 * math.pi * Julian_Day / 365)

calculate_dr = calculate_dr(Julian_Day)
print(calculate_dr)


0.9766153196172728


In [33]:
def calculate_ra(calculate_dr, calculate_ws_hs, lat_rad, calculate_delta):
    return (24 * 60 / math.pi) * 0.082 * calculate_dr * ((calculate_ws_hs * math.sin(lat_rad) * math.sin(calculate_delta)) +
                                               (math.cos(lat_rad) * math.cos(calculate_delta) * math.sin(calculate_ws_hs)))

Ra = calculate_ra(calculate_dr, calculate_ws_hs, lat_rad, calculate_delta)
print(Ra)

38.217355275896374


In [34]:
def calculate_rs(n4, l4):
    return (0.25 + 0.5 * n4) * l4
Rs = calculate_rs(nM, Ra)
print(Rs)

21.176211421068036


In [35]:
def calculate_vpd(p4, q4, s4):
    return 0.5 * (p4 + q4) - s4

VPD = calculate_vpd(calculate_e_tmax, calculate_e_tmin, calculate_ea)
print(VPD)

0.49692416847784093


In [36]:
def calculate_rn( o4, u4):
    return (1 - 0.23) * o4 - u4

Rn = calculate_rn(Rs,calculate_rb)
print(Rn)

12.755616046704823


In [37]:
def calculate_et(x4, v4, t4, h4):
    return (x4 * v4 + 0.5 * t4 * h4) / 2.45 / (x4 + 0.067 * (1 + 0.33 * h4))

ET = calculate_et(delta_value, Rn, VPD, WS)
print(f'ET value is: {ET:.2f}')


ET value is: 3.59
